In [1]:
from scipy.signal import argrelextrema

def calc_scalogram(data, scales):
    S,scales= scalogramCWT(data,scales)
    return S
def show_scalo(data, scales, colors):
    for i in range (0, len(data)):
        plt.plot(scales, data[i], colors[i], lw=3)
    plt.show()
    
# Get top X spikes from scalogram, sorted by value
def get_spikes(scalo, comparator):
    len(scalo)
    spikes = deque([(-1,-1)] * 5, maxlen=5)
    #aux = argrelextrema(scalo, comparator, order=int(len(scalo)/10))
    aux = argrelextrema(scalo, comparator)
    if aux[0].size:
        for x in np.nditer(aux) or []:
            spikes.append((scalo[x], scales[x]))
    ordered = sorted(spikes, key=lambda x: x[1], reverse=True)
    values = np.hstack(zip(*ordered))
    return values

In [2]:
from itertools import groupby
from scipy import stats
def get_stats_numpy(data, zero):
    mean = np.mean(data)
    median = np.median(data)
    std = np.std(data)
    var = np.var(data)
    skew = stats.skew(data)
    kurt = stats.kurtosis(data)
    pc = [25,50,75,90]
    percentiles = np.array(np.percentile(data, pc))
    silences = np.count_nonzero(np.asarray(data)==zero)
    silence_mean = np.mean(list(sum(1 for _ in g) for k, g in groupby(data) if k==zero))
    longest_silence = max(sum(1 for _ in g) for k, g in groupby(data) if k==zero) if silences > 0 else 0
    shortest_silence = min(sum(1 for _ in g) for k, g in groupby(data) if k==zero) if silences > 0 else 0
    
    #print("Mean: " + str(mean))
    #print("Media: " + str(median))
    #print("StdDev: " + str(std))
    #print("Variance: " + str(var))
    #print("Skewness: " + str(skew))
    #print("Kurtosis: " + str(kurt))
    #print("Pc25: " + str(percentiles[0]))
    #print("Pc50: " + str(percentiles[1]))
    #print("Pc75: " + str(percentiles[2]))
    
    features = np.hstack((mean, median, std, var, skew, kurt, percentiles, silences, silence_mean, longest_silence, shortest_silence))
    return features

In [3]:
# Put it all on a numpy array
def get_features_numpy(info, local_max_up_bytes, local_min_up_bytes, local_max_up_packet, local_min_up_packet, 
                       local_max_down_bytes, local_min_down_bytes, local_max_down_packet, local_min_down_packet, zeros):
    np.set_printoptions(suppress=True)

    result = []

    for idx in range(0, len(info)):
        result.append(
            np.hstack(
                (
                 get_stats_numpy(info[idx]['up']['byte_count'], zeros[1]),
                 get_stats_numpy(info[idx]['up']['packet_count'], zeros[0]),
                 local_max_up_bytes[idx], local_min_up_bytes[idx],
                 local_max_up_packet[idx], local_min_up_packet[idx],
                 get_stats_numpy(info[idx]['down']['byte_count'], zeros[3]),
                 get_stats_numpy(info[idx]['down']['packet_count'], zeros[2]),
                 local_max_down_bytes[idx], local_min_down_bytes[idx],
                 local_max_down_packet[idx], local_min_down_packet[idx],

            ))
        )
    return result


In [4]:
def get_info(data):
    info = []
    up = defaultdict(list)
    down = defaultdict(list)
    result = {}
    count = 0
    for second in data:
        up['packet_count'].append(second[0])
        up['byte_count'].append(second[1])
        down['packet_count'].append(second[2])
        down['byte_count'].append(second[3])
        count+=1
        if count >= sample_size:
            result['up'] = up
            result['down'] = down
            info.append(result)
            up = defaultdict(list)
            down = defaultdict(list)
            result = {}
            count = 0
    return info

In [5]:
import re

def split_number(s):
    return list(filter(None, re.split(r'(\d+)', s)))

In [6]:
import os
import numpy as np
from collections import defaultdict, deque
from scalogram import *
import pandas as pd

base_url = '../../../shared/ssh/'
#url_extension = 'dat/'
#services = ['netflix', 'youtube', 'twitch']
#windows = ['01s', '1s']
sample_size = 30 # number of intervals for each sample

for path, subdirs, files in os.walk(base_url):
    for name in files:
        print(os.path.basename(path))
        if os.path.basename(path) == 'pcap':
            continue
        
        print(os.path.join(path, name))
        
        #[up_n_packets, up_n_bytes, up_flag, down_n_packets, down_n_bytes, down_flag]
        data = np.loadtxt(os.path.join(path, name))
        data = np.delete(data, [2,5], 1)


        # Normalize
        
        data = np.vstack([data,[0,0,0,0]])
        
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        data = scaler.fit_transform(data)
        
        zeros = data[-1]
        data = data[:-1]
        
        # Split data into categories
        info = get_info(data)

        #Scalogram
        N = sample_size
        dj=1/128
        s0=2
        J=1/dj * np.log2(0.5*N/s0)
        scales=s0*2**(np.arange(J)*dj)

        scalos_up = []
        scalos_down = []
        for idx, sample in enumerate(info):
                scalos_up.append(
                    (calc_scalogram(np.asarray(sample['up']['byte_count']), scales),
                     calc_scalogram(np.asarray(sample['up']['packet_count']), scales))
                )
                scalos_down.append(
                    (calc_scalogram(np.asarray(sample['down']['byte_count']), scales),
                     calc_scalogram(np.asarray(sample['down']['packet_count']), scales))
                )

        local_max_up_bytes = []
        local_min_up_bytes = []
        local_max_up_packet = []
        local_min_up_packet = []
        local_max_down_bytes = []
        local_min_down_bytes = []
        local_max_down_packet = []
        local_min_down_packet = []

        for scalo in scalos_up:
            local_max_up_bytes.append(get_spikes(scalo[0], np.greater))
            local_min_up_bytes.append(get_spikes(scalo[0], np.less))
            local_max_up_packet.append(get_spikes(scalo[1], np.greater))
            local_min_up_packet.append(get_spikes(scalo[1], np.less))

        for scalo in scalos_down:
            local_max_down_bytes.append(get_spikes(scalo[0], np.greater))
            local_min_down_bytes.append(get_spikes(scalo[0], np.less))
            local_max_down_packet.append(get_spikes(scalo[1], np.greater))
            local_min_down_packet.append(get_spikes(scalo[1], np.less))

        # Put it in a pandas Dataframe and save it to csv
        samples = get_features_numpy(info, local_max_up_bytes, local_min_up_bytes, local_max_up_packet, local_min_up_packet,
                                    local_max_down_bytes, local_min_down_bytes, local_max_down_packet, local_min_down_packet,
                                    zeros)


        names = [
            'up_bytes_mean', 'up_bytes_median', 'up_bytes_std', 'up_bytes_var', 'up_bytes_skew', 'up_bytes_kurt',
            'up_bytes_perc25', 'up_bytes_perc50', 'up_bytes_perc75', 'up_bytes_perc90',
            'up_bytes_silences', 'up_bytes_silence_mean', 'up_bytes_longest_silence', 'up_bytes_shortest_silence',
            'up_packet_mean', 'up_packet_median', 'up_packet_std', 'up_packet_var', 'up_packet_skew', 'up_packet_kurt',
            'up_packet_perc25', 'up_packet_perc50', 'up_packet_perc75', 'up_packet_perc90',
            'up_packet_silences', 'up_packet_silence_mean', 'up_packet_longest_silence', 'up_packet_shortest_silence',
            'up_bytes_1max_y', 'up_bytes_2max_y', 'up_bytes_3max_y', 'up_bytes_4max_y', 'up_bytes_5max_y',
            'up_bytes_1max_x', 'up_bytes_2max_x', 'up_bytes_3max_x', 'up_bytes_4max_x', 'up_bytes_5max_x',
            'up_bytes_1min_y', 'up_bytes_2min_y', 'up_bytes_3min_y', 'up_bytes_4min_y', 'up_bytes_5min_y',
            'up_bytes_1min_x', 'up_bytes_2min_x', 'up_bytes_3min_x', 'up_bytes_4min_x', 'up_bytes_5min_x',
            'up_packet_1max_y', 'up_packet_2max_y', 'up_packet_3max_y', 'up_packet_4max_y', 'up_packet_5max_y',
            'up_packet_1max_x', 'up_packet_2max_x', 'up_packet_3max_x', 'up_packet_4max_x', 'up_packet_5max_x',
            'up_packet_1min_y', 'up_packet_2min_y', 'up_packet_3min_y', 'up_packet_4min_y', 'up_packet_5min_y',
            'up_packet_1min_x', 'up_packet_2min_x', 'up_packet_3min_x', 'up_packet_4min_x', 'up_packet_5min_x',

            'down_bytes_mean', 'down_bytes_median', 'down_bytes_std', 'down_bytes_var', 'down_bytes_skew', 'down_bytes_kurt',
            'down_bytes_perc25', 'down_bytes_perc50', 'down_bytes_perc75', 'down_bytes_perc90',
            'down_bytes_silences', 'down_bytes_silence_mean', 'down_bytes_longest_silence', 'down_bytes_shortest_silence',
            'down_packet_mean', 'down_packet_median', 'down_packet_std', 'down_packet_var', 'down_packet_skew', 'down_packet_kurt',
            'down_packet_perc25', 'down_packet_perc50', 'down_packet_perc75', 'down_packet_perc90',
            'down_packet_silences', 'down_packet_silence_mean', 'down_packet_longest_silence', 'down_packet_shortest_silence',
            'down_bytes_1max_y', 'down_bytes_2max_y', 'down_bytes_3max_y', 'down_bytes_4max_y', 'down_bytes_5max_y',
            'down_bytes_1max_x', 'down_bytes_2max_x', 'down_bytes_3max_x', 'down_bytes_4max_x', 'down_bytes_5max_x',
            'down_bytes_1min_y', 'down_bytes_2min_y', 'down_bytes_3min_y', 'down_bytes_4min_y', 'down_bytes_5min_y',
            'down_bytes_1min_x', 'down_bytes_2min_x', 'down_bytes_3min_x', 'down_bytes_4min_x', 'down_bytes_5min_x',
            'down_packet_1max_y', 'down_packet_2max_y', 'down_packet_3max_y', 'down_packet_4max_y', 'down_packet_5max_y',
            'down_packet_1max_x', 'down_packet_2max_x', 'down_packet_3max_x', 'down_packet_4max_x', 'down_packet_5max_x',
            'down_packet_1min_y', 'down_packet_2min_y', 'down_packet_3min_y', 'down_packet_4min_y', 'down_packet_5min_y',
            'down_packet_1min_x', 'down_packet_2min_x', 'down_packet_3min_x', 'down_packet_4min_x', 'down_packet_5min_x'
        ]
    
        df = pd.DataFrame(samples, columns=names)
        outdir = 'csv/' + str(sample_size) + 's' + '.'.join(name.split('_')[1].split('.')[:-1]) + '/' + split_number(name)[0] + '/'        
        outname =  name.split('.')[0] + '.csv'
        if not os.path.exists(outdir):
            os.mkdir(outdir)
        df['label'] = split_number(name)[0]
        df.to_csv(os.path.join(outdir, outname), sep=',', encoding='utf-8')
        

dat
../../../shared/ssh/netflix/dat/netflix-ssh3.pcapng_0.10s.dat


/home/agro/work/thesis/env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/agro/work/thesis/env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


dat
../../../shared/ssh/netflix/dat/netflix-ssh3.pcapng_1.00s.dat
pcap
dat
../../../shared/ssh/youtube/dat/youtube-ssh1.pcapng_1.00s.dat
dat
../../../shared/ssh/youtube/dat/youtube-ssh1.pcapng_0.10s.dat
pcap
pcap


In [7]:
df

,up_bytes_mean,up_bytes_median,up_bytes_std,up_bytes_var,up_bytes_skew,up_bytes_kurt,up_bytes_perc25,up_bytes_perc50,up_bytes_perc75,up_bytes_perc90,...,down_packet_2min_y,down_packet_3min_y,down_packet_4min_y,down_packet_5min_y,down_packet_1min_x,down_packet_2min_x,down_packet_3min_x,down_packet_4min_x,down_packet_5min_x,label
0,-0.213556,-0.215238,9.059638e-03,8.207704e-05,5.199469,25.034483,-0.215238,-0.215238,-0.215238,-0.215238,...,0.009478,0.007315,-1.000000,-1.000000,12.337687,8.771433,2.693884,-1.000000,-1.000000,youtube-ssh
1,-0.189162,-0.215238,1.404244e-01,1.971901e-02,5.199469,25.034483,-0.215238,-0.215238,-0.215238,-0.215238,...,0.008846,0.008561,0.009110,-1.000000,12.539757,8.819061,6.202340,3.648207,-1.000000,youtube-ssh
2,0.705502,0.067394,1.517344e+00,2.302332e+00,1.818255,1.940202,-0.215238,0.067394,0.279367,3.205617,...,0.014941,0.001089,-1.000000,-1.000000,12.271056,8.915094,3.570033,-1.000000,-1.000000,youtube-ssh
3,0.867511,0.185998,1.325555e+00,1.757096e+00,1.580317,2.843429,-0.215238,0.185998,1.532285,2.414752,...,0.005479,0.001682,-1.000000,-1.000000,10.775534,4.860989,2.497202,-1.000000,-1.000000,youtube-ssh
4,-0.163423,-0.215238,1.040378e-01,1.082387e-02,1.531213,0.403267,-0.215238,-0.215238,-0.215238,0.037112,...,0.005430,0.006666,-1.000000,-1.000000,12.953881,7.375866,4.177095,-1.000000,-1.000000,youtube-ssh
5,-0.009657,-0.215238,4.259278e-01,1.814145e-01,2.127415,3.347358,-0.215238,-0.215238,-0.164768,0.695240,...,0.011950,0.004095,-1.000000,-1.000000,12.676306,9.110309,4.604757,-1.000000,-1.000000,youtube-ssh
6,-0.047342,-0.215238,4.898742e-01,2.399767e-01,3.008061,7.568691,-0.215238,-0.215238,-0.215238,0.120387,...,0.012477,0.000826,-1.000000,-1.000000,12.073315,8.676947,4.268562,-1.000000,-1.000000,youtube-ssh
7,-0.215238,-0.215238,2.775558e-17,7.703720e-34,1.000000,-2.000000,-0.215238,-0.215238,-0.215238,-0.215238,...,0.012208,-1.000000,-1.000000,-1.000000,12.337687,8.724062,-1.000000,-1.000000,-1.000000,youtube-ssh
8,-0.182433,-0.215238,1.346628e-01,1.813407e-02,4.361919,18.257531,-0.215238,-0.215238,-0.215238,-0.215238,...,0.009971,-1.000000,-1.000000,-1.000000,12.337687,8.771433,-1.000000,-1.000000,-1.000000,youtube-ssh
9,2.398938,-0.000741,6.658980e+00,4.434201e+01,3.636263,12.500663,-0.215238,-0.000741,1.812393,3.410020,...,0.003423,0.005143,-1.000000,-1.000000,12.676306,8.963502,3.291511,-1.000000,-1.000000,youtube-ssh
